In [1]:
import sys
sys.path.insert(0, '../../style_generation_pipeline')

In [2]:
import pandas as pd
import json
import sklearn
import glob
import pickle
from sklearn.model_selection import train_test_split

pd.set_option('display.width', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
from data import *

In [4]:
path='/mnt/swordfish-pool2/milad/hiatus-data/phase_2'

### Compuate correlation:

In [45]:
llm_feats_df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/filtered/refined_and_aggregated_features_final.csv')
gram2vec_feats_df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/gram2vec_feats.csv')

In [46]:
total_num_documents = llm_feats_df.documentID.nunique()

In [49]:
## Compute co-occurancies between pairs of feats
llm_feats_df = llm_feats_df.groupby('final_attribute_name').agg({'documentID': lambda x: list(x)}).reset_index()
gram2vec_feats_df = gram2vec_feats_df.groupby('gram2vec_feats').agg({'documentID': lambda x: list(x)}).reset_index()

In [50]:
llm_feats_dict = {x[0]: x[1] for x in zip(llm_feats_df.final_attribute_name.tolist(), llm_feats_df.documentID.tolist())}
g2v_feats_dict = {x[0]: x[1] for x in zip(gram2vec_feats_df.gram2vec_feats.tolist(), gram2vec_feats_df.documentID.tolist())}

In [63]:
occ_matrix = np.zeros(shape=(len(g2v_feats_dict.keys()), len(llm_feats_dict.keys())))

In [64]:
for i, g2v_feat in enumerate(g2v_feats_dict.keys()):
    for j, llm_feat in enumerate(llm_feats_dict.keys()):
        g2v_docs, llm_docs = g2v_feats_dict[g2v_feat], llm_feats_dict[llm_feat]
        prop_g2v  = len(g2v_docs)/total_num_documents
        prop_llm  = len(llm_docs)/total_num_documents
        prop_both = len(set(g2v_docs).intersection(set(llm_docs)))/total_num_documents
        occ_matrix[i,j] = prop_both/prop_llm

In [67]:
occ_matrix_df = pd.DataFrame(occ_matrix, columns=llm_feats_dict.keys())
occ_matrix_df['g2v_feat'] = g2v_feats_dict.keys()

In [69]:
occ_matrix_df.to_csv(path + '/g2v_and_llm_occ_matrix.csv', index=False)

### Analysing the interp spaces:

In [90]:
#Load gram2vec and llm-based features
llm_feats_df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/filtered/refined_and_aggregated_features_final.csv')
gram2vec_feats_df = pd.read_csv('/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/gram2vec_feats.csv')
llm_feats = llm_feats_df.final_attribute_name.unique().tolist()
gram2vec_feats = gram2vec_feats_df.gram2vec_feats.unique().tolist()

In [91]:
#load the interpretable spaces
interp_spaces = {
    'gram2vec': ['/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/gram2vec'],
    'llm': ['/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/llm/'],
    'both': ['/mnt/swordfish-pool2/milad/hiatus-data/phase_2/explainability/llm_and_gram2vec'],
}

for key, val in interp_spaces.items():
    # style-distance clustering
    interp_space = pd.read_pickle(val[0] + "/interpretable_space.pkl")
    clustering_df  = pd.read_pickle(val[0] + "/train_authors_documents.pkl")
    # DBSCAN generate a cluster -1 of all outliers. We don't want this cluster
    del interp_space[-1]
    clustering_df = clustering_df[clustering_df.cluster_label != -1]

    cluster_documents_df = clustering_df.groupby('cluster_label').agg({'documentID': lambda x: len(list(x))}).reset_index()
    interp_space_df = pd.DataFrame([{'cluster_label': x[0], 'style_feats': sorted(x[1][1].items(), key=lambda x:-x[1])} for x in interp_space.items()])
    interp_space_df['num_feats'] = interp_space_df.style_feats.apply(lambda x: len(x))
    interp_space_df['num_documents'] = interp_space_df.cluster_label.apply(lambda x: cluster_documents_df[cluster_documents_df.cluster_label==x]['documentID'].tolist()[0])

    interp_spaces[key].append(interp_space)
    interp_spaces[key].append(clustering_df)
    interp_spaces[key].append(interp_space_df)

In [92]:
all_in_one_interp_space = {}
for c_label in interp_spaces['llm'][-1].cluster_label.tolist():
    llm_feats  = interp_spaces['llm'][-1][interp_spaces['llm'][-1].cluster_label == c_label]['style_feats'].tolist()[0]
    g2v_feats  = interp_spaces['gram2vec'][-1][interp_spaces['gram2vec'][-1].cluster_label == c_label]['style_feats'].tolist()[0]
    both_feats = interp_spaces['both'][-1][interp_spaces['both'][-1].cluster_label == c_label]['style_feats'].tolist()[0]
    all_in_one_interp_space[c_label] = {
        'ranked-llm-feats'  : [x[0] for x in llm_feats],
        'ranked-g2v-feats'  : [x[0] for x in g2v_feats],
        'ranked-both-feats' : [x[0] for x in both_feats]
    }
json.dump(all_in_one_interp_space, open(path + '/explainability/interp_space_style_representations.json', 'w'))

In [93]:
#find the source of top features in the representation
g2v_appearing = 0
llm_appearing = 0
top_k=10
for c_label, rep in all_in_one_interp_space.items():
    top_k_feats = rep['ranked-both-feats'][:top_k]
    print(top_k_feats)
    top_k_feats_sources = ['g2v' if f in gram2vec_feats else 'llm' for f in top_k_feats]
    print(top_k_feats_sources)
    g2v_appearing+= top_k_feats_sources.count('g2v')
    llm_appearing+= top_k_feats_sources.count('llm')

print('llm:', llm_appearing/(10*len(all_in_one_interp_space.keys())), 'g2v:', g2v_appearing/(10*len(all_in_one_interp_space.keys())))

['Part-of-Speech Unigram:Other (foreign words, typos, abbreviations)', 'Part-of-Speech Unigram:Proper noun', 'Part-of-Speech Bigram:Proper noun followed by Other', 'tense usage', 'style is formal', 'precise language', 'Morphology Tag:Comma punctuation type', 'more information', 'additional clauses', 'Dependency Label:Object of preposition']
['g2v', 'g2v', 'g2v', 'llm', 'llm', 'llm', 'g2v', 'llm', 'llm', 'g2v']
['passive voice', 'overall rhythm', 'complex ideas', 'cohesive narrative', 'vivid language', 'better engagement', 'Dependency Label:Object of preposition', 'rhetorical devices', 'dynamic narrative flow', 'conceptual ideas']
['llm', 'llm', 'llm', 'llm', 'llm', 'llm', 'g2v', 'llm', 'llm', 'llm']
['improved writing', 'dynamic style', 'Part-of-Speech Unigram:Adverb', 'unique word selections', 'casual tone', 'strong visuals', 'hearted atmosphere', 'style is informal', 'literary devices', 'tone is positive']
['llm', 'llm', 'g2v', 'llm', 'llm', 'llm', 'llm', 'llm', 'llm', 'llm']
['compl

In [94]:
interp_spaces['gram2vec'][-1].head()

,cluster_label,style_feats,num_feats,num_documents
0,0,"[(Part-of-Speech Unigram:Other (foreign words, typos, abbreviations), 131.02775417812242), (Part-of-Speech Unigram:Proper noun, 120.37283153046812), (Part-of-Speech Bigram:Proper noun followed by Other, 102.20907778184412), (Morphology Tag:Comma punctuation type, 55.79294317523215), (Dependency Label:Object of preposition, 46.84675719527894), (Part-of-Speech Bigram:Other followed by Proper noun, 42.02924258041938), (Part-of-Speech Bigram:Other followed by Punctuation, 39.20883457530391), (Dependency Label:Appositional modifier, 34.478090413118295), (punctuation:,, 33.43863491498796), (Dependency Label:Prepositional modifier, 24.66571461542978), (Part-of-Speech Unigram:Adposition (preposition or postposition), 19.997055686634315), (Part-of-Speech Unigram:Punctuation, 15.388914333349422), (Dependency Label:Noun phrase as adverbial modifier, 14.902192988849615), (Part-of-Speech Bigram:Other followed by Other, 11.785903752756516), (Dependency Label:Punctuation, 10.809595623388658), (Part-of-Speech Bigram:Proper noun followed by Proper noun, 9.486092586645196), (Part-of-Speech Bigram:Punctuation followed by Other, 8.837390855544697), (Morphology Tag:Third person, 6.972482233897174), (Dependency Label:Nominal modifier, 6.004177511488482), (Morphology Tag:Past tense, 4.694256129153166), (Part-of-Speech Bigram:Noun followed by Punctuation, 3.529123158143493), (Part-of-Speech Unigram:Verb, 2.6354238904222647), (Part-of-Speech Unigram:Pronoun, 1.5749697240010292), (Morphology Tag:Personal pronoun type, 1.505021649615636), (Part-of-Speech Unigram:Noun, 0.6339761166199728), (Dependency Label:Nominal subject, 0.36624160262986766), (Morphology Tag:Singular number, 0.020477816023608483)]",27,47
1,1,"[(Dependency Label:Object of preposition, 8.198182509173813), (Part-of-Speech Unigram:Adverb, 6.864553609200628), (Morphology Tag:Positive degree, 6.576629541298957), (Dependency Label:Adverbial modifier, 6.539921988538429), (Part-of-Speech Unigram:Adjective, 6.391763000506573), (Dependency Label:Adjectival modifier, 5.789182960835995), (Dependency Label:Prepositional modifier, 5.6920879881761035), (Part-of-Speech Unigram:Determiner, 4.271459574169029), (Part-of-Speech Unigram:Adposition (preposition or postposition), 4.209906460344066), (Morphology Tag:Article pronoun type, 4.017109289939661), (Dependency Label:Determiner, 3.733700886050131), (Part-of-Speech Bigram:Determiner followed by Noun, 3.206179073723333), (Part-of-Speech Unigram:Punctuation, 2.946813382981804), (Morphology Tag:Present tense, 2.915812435900883), (Part-of-Speech Unigram:Proper noun, 2.616800685444959), (Morphology Tag:Third person, 2.3241607446323913), (Dependency Label:Punctuation, 2.0699225661808067), (Part-of-Speech Unigram:Auxiliary verb, 1.891376864445471), (Part-of-Speech Unigram:Verb, 1.3177119452111323), (Dependency Label:Nominal subject, 1.098724807889603), (Part-of-Speech Unigram:Pronoun, 1.0499798160006861), (Part-of-Speech Unigram:Noun, 0.18847938602215406), (Morphology Tag:Singular number, 0.004792680345950921)]",23,11
2,2,"[(Part-of-Speech Unigram:Adverb, 8.580692011500785), (Morphology Tag:Positive degree, 6.576629541298957), (Dependency Label:Adverbial modifier, 6.539921988538429), (Dependency Label:Determiner, 6.533976550587729), (Part-of-Speech Unigram:Determiner, 5.980043403836641), (Dependency Label:Object of preposition, 5.8558446494098675), (Dependency Label:Prepositional modifier, 5.6920879881761035), (Part-of-Speech Bigram:Noun followed by Adposition, 5.470095025558225), (Part-of-Speech Unigram:Adjective, 5.113410400405258), (Part-of-Speech Unigram:Adposition (preposition or postposition), 4.736144767887074), (Morphology Tag:Present tense, 2.915812435900883), (Dependency Label:Adjectival modifier, 2.8945914804179975), (Dependency Label:Direct object, 2.891970246938123), (Part-of-Speech Unigram:Other (foreign words, typos, abbreviations), 2.848429438654835), (Part-of-Speech Unigram:Punctuation, 

In [95]:
interp_spaces['llm'][-1].head()

,cluster_label,style_feats,num_feats,num_documents
0,0,"[(tense usage, 75.39465300649763), (style is formal, 74.2656332899042), (precise language, 64.89579206072429), (more information, 53.21230144554308), (additional clauses, 50.54654731343905), (professional tone, 45.220097300138846), (effective communication, 43.75176563490477), (formal manner, 43.307045578469875), (clear sentence style, 41.348012969058935), (various sentence structures, 39.01428002735918), (methodical approach, 36.13903865732184), (parallel structures, 34.69948778854563), (diverse sentence structures, 30.563382515793197), (transitional elements, 29.983581266085498), (consistent tone, 26.554425596411537), (subordinate clauses, 23.95746896636315), (interpersonal relationships, 23.215900065691077), (possessive pronouns, 22.02016453489306), (style is professional, 21.362952329727957), (transitional phrases effectively, 21.136458524011243), (extra details, 20.5002638677571), (transitional phrases, 19.918756484602568), (previous events, 19.057016982331408), (extra information, 17.924861079259195), (different sentence structures, 17.3474628812382), (relative clauses, 17.176173063101245), (more context, 16.737447317630963), (better understanding, 16.62422633368242), (rhythmic impact, 16.308144786708944), (tone is contemplative, 15.678983193762733), (rhetorical devices, 15.488973612870613), (diverse information, 14.090972349340829), (verb tense, 14.090972349340829), (passive voice, 13.803324277842982), (comprehensive summary, 13.515608204437266), (emotional level, 13.2800421331245), (structured manner, 11.785613329464057), (literary devices, 11.718512771362786), (structured approach, 11.492406381080306), (advanced sentence structures, 11.403502855938639), (overall impact, 11.318383627101047), (various sentence structures effectively, 11.158298211753975), (different lengths, 11.01008226744653), (particular structure, 10.968529304736109), (diverse sentence lengths, 10.87855215591072), (direct engagement, 9.623787906326639), (varied sentence structures, 9.325716748005664), (formal language, 8.484251587527758), (figurative language, 8.202114773173552), (overall flow, 7.738633355230359), (specific words, 7.738633355230359), (additional details, 7.738633355230359), (sentence is complex, 7.738633355230359), (chronological narrative structure, 7.738633355230359), (different methods, 7.286150558516912), (important information, 7.2278077314643685), (various connectors, 7.20160736213704), (multiple elements, 7.0454861746704145), (overall rhythm, 6.936647958165022), (selective use, 6.8913354948431556), (interpersonal connections, 6.8913354948431556), (detailed language, 6.757804102218633), (style is polite, 6.439350371100098), (casual language, 6.3578122485224915), (detailed information, 6.064656921658687), (detailed descriptions, 5.659191813550524), (simple language structure, 5.579149105876987), (complex sentence structures, 5.519659081534761), (logical flow, 5.436048262236314), (parallel structure, 5.371509741098743), (structured word order, 5.148366189784533), (improved flow, 5.123673577194161), (rhythmic effect, 5.076045528204907), (specific pattern, 5.008604247409374), (additional information, 4.951886332650948), (descriptive language, 4.744963108934412), (accessible language, 4.742901081676369), (consistent verb tense, 4.734388547404105), (formal style, 4.480536817208877), (smooth connections, 4.44279648922603), (smooth transitions, 4.44279648922603), (elicit emotions, 4.418405036101871), (simple sentence structures, 4.405224572073567), (style is clear, 4.164416809436563), (different ideas, 4.136765278106052), (friendly tone, 4.058122150786939), (better rhythm, 4.033224599165212), (formal tone, 3.3483079177417827), (neutral tone, 3.335987433353742), (different techniques, 2.9237426382601988)]",90,47
1,1,"[(passive voice, 13.803324277842982), (overall rhythm, 11.561079930275035), (complex ideas, 10.197499004925822), (cohesive narrative, 9.810840022348287), (vivid languag

In [96]:
interp_spaces['both'][-1].head()

cluster_label  \
0              0   
1              1   
2              2   
3              3   
4              4   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          